[CRF code]('https://sklearn-crfsuite.readthedocs.io/en/latest/tutorial.html')

[CRF介紹]('http://blog.echen.me/2012/01/03/introduction-to-conditional-random-fields/')

#CRF

In [ ]:
!pip install sklearn-crfsuite

     |████████████████████████████████| 743 kB 3.2 MB/s 


In [ ]:
import nltk
import sklearn
import scipy.stats
import sklearn_crfsuite
from itertools import chain  
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
nltk.download('conll2002')

[nltk_data] Downloading package conll2002 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2002.zip.


True

In [ ]:
nltk.corpus.conll2002.fileids()

['esp.testa', 'esp.testb', 'esp.train', 'ned.testa', 'ned.testb', 'ned.train']

In [ ]:
train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))

In [ ]:
print(len(train_sents))
print(len(test_sents))

8323
1517


[種類介紹]('https://chriskhanhtran.github.io/posts/named-entity-recognition-with-transformers/')
在網站中的 3.label 部分

In [ ]:
train_sents[0] # ('單字','詞性','種類')

[('Melbourne', 'NP', 'B-LOC'),
 ('(', 'Fpa', 'O'),
 ('Australia', 'NP', 'B-LOC'),
 (')', 'Fpt', 'O'),
 (',', 'Fc', 'O'),
 ('25', 'Z', 'O'),
 ('may', 'NC', 'O'),
 ('(', 'Fpa', 'O'),
 ('EFE', 'NC', 'B-ORG'),
 (')', 'Fpt', 'O'),
 ('.', 'Fp', 'O')]

##標註特徵 label feature

In [ ]:
def word2features(sent, i):
  word = sent[i][0]
  postag = sent[i][1]

  features = {              # 基本的單字資料
      'bias': 1.0,
      'word.lower()': word.lower(),    # 縮寫的單字
      'word[-3:]': word[-3:],       # 字尾的三個字母
      'word[-2:]': word[-2:],       # 字尾的兩個字母
      'word.isupper()': word.isupper(),  # 判斷是否為大寫
      'word.istitle()': word.istitle(),  # 判斷是否為標題
      'word.isdigit()': word.isdigit(),  # 判斷是否為數字
      'postag': postag,
      'postag[:2]': postag[:2],    # tag字首的兩個單字
  }
  if i > 0:                # 前面有字時 >>> 加入前面單字的情況
      word1 = sent[i-1][0]
      postag1 = sent[i-1][1]
      features.update({
          '-1:word.lower()': word1.lower(),
          '-1:word.istitle()': word1.istitle(),
          '-1:word.isupper()': word1.isupper(),
          '-1:postag': postag1,
          '-1:postag[:2]': postag1[:2],
      })
  else:
      features['BOS'] = True

  if i < len(sent)-1:           # 後面有字時 >>> 加入後面單字的情況
      word1 = sent[i+1][0]
      postag1 = sent[i+1][1]
      features.update({
          '+1:word.lower()': word1.lower(),
          '+1:word.istitle()': word1.istitle(),
          '+1:word.isupper()': word1.isupper(),
          '+1:postag': postag1,
          '+1:postag[:2]': postag1[:2],
      })
  else:
      features['EOS'] = True

  return features

In [ ]:
def sent2features(sent):
  return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
  return [label for token, postag, label in sent]

def sent2tokens(sent):
  return [token for token, postag, label in sent]

In [ ]:
# 計算執行時間
%%time 

X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

CPU times: user 1.05 s, sys: 173 ms, total: 1.22 s
Wall time: 1.22 s


## model

In [ ]:
%%time
crf = sklearn_crfsuite.CRF(     
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CPU times: user 38.5 s, sys: 201 ms, total: 38.7 s
Wall time: 41.1 s


## result

[檢測結果的方法]('https://sklearn-crfsuite.readthedocs.io/en/latest/api.html') :
accuracy , precision , recall , F1-score

F1 score

In [ ]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred,average='weighted', labels=labels)

0.7964686316443963

accuracy

In [ ]:
sklearn_crfsuite.metrics.flat_accuracy_score(y_test, y_pred)

0.971455184056818